In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve, validation_curve
%matplotlib
import matplotlib.pyplot as plt
import numpy as np

Using matplotlib backend: Qt5Agg


In [2]:
# 添加列名
header_row = ['age','sex','cp','trestbps','chol','fbs',\
                'restecg','thalach','exang','oldpeak','slope','ca','thal','target']
 
# 载入数据
heart = pd.read_csv('processed.cleveland.data.csv',names=header_row)

In [3]:
heart

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [4]:
x=heart[['age','sex','cp','trestbps','chol','fbs',\
                'restecg','thalach','exang','oldpeak','slope','ca','thal']]
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7


In [5]:

#心脏病（0=否，1=是）
heart["target"] = np.where(heart["target"] != 0, 1, 0)
y=heart.target

In [6]:
y

0      0
1      1
2      1
3      0
4      0
      ..
292    1
293    1
294    1
295    1
296    1
Name: target, Length: 297, dtype: int32

In [7]:
#分割数据集到训练集和数据集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)
#进行处理（特征工程）one hot 编码
dict=DictVectorizer(sparse=False)
x_train=dict.fit_transform(x_train.to_dict(orient="records"))
print(dict.get_feature_names())
x_test=dict.transform(x_test.to_dict(orient="records"))
print(x_train)


['age', 'ca', 'chol', 'cp', 'exang', 'fbs', 'oldpeak', 'restecg', 'sex', 'slope', 'thal', 'thalach', 'trestbps']
[[ 64.   1. 263. ...   7. 105. 128.]
 [ 55.   1. 289. ...   7. 145. 160.]
 [ 57.   1. 335. ...   7. 143. 110.]
 ...
 [ 58.   0. 340. ...   3. 172. 120.]
 [ 63.   0. 252. ...   3. 172. 135.]
 [ 35.   0. 198. ...   7. 130. 120.]]


In [8]:
#用决策树进行预测
dec=DecisionTreeClassifier(max_depth=6,min_samples_leaf=5,min_samples_split=2)
dec.fit(x_train,y_train)

#####20201023
train_size, train_scores, test_scores = learning_curve(dec, x_train, y_train, cv=10, scoring='accuracy',#10折交叉验证
                                                       train_sizes = np.linspace(0.1,1.0,10))#5次的训练数量占比
mean_train = np.mean(train_scores,1)  #(5,)
# 得到得分范围的上下界
upper_train = np.clip(mean_train + np.std(train_scores,1),0,1) 
lower_train = np.clip(mean_train - np.std(train_scores,1),0,1)
    
mean_test = np.mean(test_scores,1)
# 得到得分范围的上下界
upper_test = np.clip(mean_test + np.std(test_scores,1),0,1) 
lower_test = np.clip(mean_test - np.std(test_scores,1),0,1)



plt.figure('Fig1')
plt.plot(train_size,mean_train,'ro-',label='train score')
plt.plot(train_size,mean_test,'go-',label='test score')
##填充上下界的范围
plt.fill_between(train_size,upper_train,lower_train,alpha=0.2,#alpha：覆盖区域的透明度[0,1],其值越大，表示越不透明 
         color='r')                   
plt.fill_between(train_size,upper_test,lower_test,alpha=0.2,#alpha：覆盖区域的透明度[0,1],其值越大，表示越不透明 
         color='g')  
plt.grid()
plt.xlabel('train size')
plt.ylabel('score')
plt.legend(loc='lower right')
plt.title('Decision Trees')
plt.savefig('train number-size.png')
plt.show()


In [9]:
#用随机森林进行预测(超参数调优)
rf=RandomForestClassifier()

param={"n_estimators":[120,200,300,500,800,1200],"max_depth":[5,8,15,25,30]}
#网格搜索与交叉验证
gc=GridSearchCV(rf,param_grid=param,cv=2)
gc.fit(x_train,y_train)

print("准确率:",gc.score(x_test,y_test))
print("选择的参数模型：",gc.best_params_)



准确率: 0.8111111111111111
选择的参数模型： {'max_depth': 30, 'n_estimators': 1200}


In [10]:
import os
os.environ["PATH"]+= os.pathsep + 'E:/Graphviz/bin'
from subprocess import call
call(['dot', '-Tpng','tree.dot', '-o', 'tree.png', '-Gdpi=600'])
 
# 在jupyter notebook中查看决策树图像
from IPython.display import Image
Image(filename ='tree.png')


In [11]:
import codecs
txt_dir = 'tree.dot'   
txt_dir_utf8 = 'tree_utf8.dot'  

with codecs.open(txt_dir, 'r',encoding='utf-8') as f, codecs.open(txt_dir_utf8, 'w', encoding='utf-8') as wf:
    for line in f:
        lines = line.strip().split('\t')
        print(lines)
        if 'label' in lines[0]:
            newline = lines[0].replace('\n', '').replace(' ', '')
        else:
            newline = lines[0].replace('\n','').replace('helvetica', ' "Microsoft YaHei" ')
        wf.write(newline + '\t')


['digraph Tree {']
['node [shape=box, style="filled, rounded", color="black", fontname=helvetica] ;']
['edge [fontname=helvetica] ;']
['0 [label=<trestbps &le; 3.5<br/>gini = 0.497<br/>samples = 207<br/>value = [112, 95]<br/>class = 0>, fillcolor="#fbece1"] ;']
['1 [label=<slope &le; 4.5<br/>gini = 0.313<br/>samples = 103<br/>value = [83, 20]<br/>class = 0>, fillcolor="#eb9f69"] ;']
['0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;']
['2 [label=<exang &le; 0.5<br/>gini = 0.131<br/>samples = 71<br/>value = [66, 5]<br/>class = 0>, fillcolor="#e78b48"] ;']
['1 -> 2 ;']
['3 [label=<ca &le; 172.5<br/>gini = 0.051<br/>samples = 38<br/>value = [37, 1]<br/>class = 0>, fillcolor="#e6843e"] ;']
['2 -> 3 ;']
['4 [label=<gini = 0.0<br/>samples = 33<br/>value = [33, 0]<br/>class = 0>, fillcolor="#e58139"] ;']
['3 -> 4 ;']
['5 [label=<gini = 0.32<br/>samples = 5<br/>value = [4, 1]<br/>class = 0>, fillcolor="#eca06a"] ;']
['3 -> 5 ;']
['6 [label=<ca &le; 160.0<br/>gini = 0.213<br/>sampl